In [ ]:
# working on a better workflow for streaming tweets

# todo save tweets to a database, not json text

In [1]:
import tweepy as twee
import json
from datetime import datetime
from dateutil.tz import tzutc
import time
import csv
from http.client import IncompleteRead
from requests.packages.urllib3.exceptions import ProtocolError
from ConfigParser import SafeConfigParser


In [2]:
# params

KEYWORD = "abrams"
pause_time = 20 # pause time in minutes when reaching REST api call limit # todo think hard about sampling
max_tweets = 10000


print twee.__version__

# read in access credentials from the private file
config = SafeConfigParser()
config.read('config.ini')
consumer_key = config.get('twitter_api', 'consumer_key')
consumer_secret = config.get('twitter_api', 'consumer_secret')
access_token = config.get('twitter_api', 'access_token')
access_token_secret = config.get('twitter_api', 'access_token_secret')

auth = twee.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = twee.API(auth)

3.6.0


In [11]:
# class to encapsulate the streaming

class MyStreamListener(twee.StreamListener):

    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open('tweet_records.json','a')
        self.maxtweets = max_tweets
        self.file.write("[")

    def on_status(self, status):
        #print([status.author.screen_name, status.created_at, status.text])
        
        if hasattr(status,'retweeted_status'):
            print 'retweet: '
            print status.text
        
        else:   # if not a retweet
            self.num_tweets += 1
            if self.num_tweets < self.maxtweets:
                timestamp = status.created_at
                # convert to utc
                if timestamp.tzinfo:
                    timestamp.astimezone(tzutc).replace(tzinfo=None)
                timestamp_serializeable = timestamp.isoformat() # serialize
                
                try:
                    tweet_text = status.extended_tweet["full_text"]
                    print("extended tweet:")
                    print(tweet_text)
                except AttributeError:
                    tweet_text = status.text
                    print("tweet:")
                    print(tweet_text)
            
                tweet_info = {"screen_name":status.author.screen_name,
                             "created_at":timestamp_serializeable,
                             "text":tweet_text} # possibly need to convert to utf-8 first
                                            # or store as json objects instead maybe? # update got this done
                    
                json.dump(tweet_info,self.file, indent=4, sort_keys=True, separators=(',', ':'))
                if self.num_tweets < (self.maxtweets-1):
                    self.file.write(",")
                print 't ' + str(self.num_tweets)
                time.sleep(1) # minimum spacing of samples
                return True
            else:
                self.close_up_shop()
                return False


        #except Exception:
        #    print "caught error: " + str(Exception)

    def on_error(self, status_code):
        print "ERR"
        print status_code
        #if status_code == 420:
        #    return False #returning False in on_data disconnects the stream
        self.close_up_shop()
        return False

    def on_limit(self, status):
        print 'Limit threshold exceeded. Pausing. ' # , status
        time.sleep(60 * pause_time) # convert minutes to seconds
        #self.close_up_shop()
        return True

    def on_timeout(self, status):
        print 'Stream disconnected...'
        self.close_up_shop()
        return False

    def close_up_shop(self):
        self.file.write("]")
        self.file.close()

In [10]:
while True:
    try:
        # Connect/reconnect the stream
        myStreamListener = MyStreamListener()
        myStream = twee.Stream(auth = api.auth, listener=myStreamListener)
        # DON'T run this approach async or you'll just create a ton of streams!
        myStream.filter(track=[KEYWORD], async=False)

    except ProtocolError:
        print "excepted protocol error"
        time.sleep(10)
        # Oh well, reconnect and keep trucking
        continue
               # wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
#    elapsed = (time.time() - start)

print "running async tasks" # todo wait is this asynchronous or not

RT @ava: Yes, Queen. Yes! Count them all! https://t.co/8BygAqJRXg
t 1
RT @RealSaavedra: This is media strategy, not political

The Democrats know she is going to lose and if they refuse to concede then the mor…
t 2
RT @cmccarron: Totally agree @tribelaw. I know I should be grateful for the House win but I am saddened to see such stellar candidates and…
t 3
RT @NBCNews: “I promise you tonight, we’re going to make sure every single vote is counted. Every single vote.” 

Stacey Abrams speaks to s…
t 4
RT @MrFilmkritik: If Stacey Abrams loses, every voting rights group in existence should sue the daylights out of Brian Kemp for suppressing…
t 5
RT @MrFilmkritik: If Stacey Abrams loses, every voting rights group in existence should sue the daylights out of Brian Kemp for suppressing…
t 6
RT @chrislhayes: Part of the reason Abrams, O’Rourke, and Gillum got a lot of national attention is that they’re all trying to do something…
t 7
RT @MrFilmkritik: If Stacey Abrams loses, every voting rights

KeyboardInterrupt: 